In [4]:
#Importing libraries
import pickle
import nltk
from gensim.models import FastText
from gensim.test.utils import common_texts
import sklearn.manifold
import numpy as np
import pandas as pd
import re
import multiprocessing
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
stop_words = set(stopwords.words('english'))
ps = PorterStemmer()
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
import matplotlib.pyplot as plt 
import seaborn as sns
import time
import collections
from scipy import stats 
import numpy as np 

In [81]:
from gensim.models import Word2Vec
from gensim.test.utils import common_texts

In [1]:
file = open("wiki.txt", "r",errors='ignore').read() 

In [2]:
print("Corpus is now {0} characters long".format(len(file)))

Corpus is now 968283996 characters long


In [5]:
#Split the corpus into sentences
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
raw_sentences = tokenizer.tokenize(file)

In [6]:
#convert into a list of words
#remove unnecessary, split into words, no hyphens
#list of words
def sentence_to_wordlist(raw):
    clean=re.sub('[\n]',' ',raw.lower())
    clean = re.sub("[^a-zA-Z]"," ", raw.lower())
    words = clean.split()
    #words = [ps.stem(w) for w in words if not w in stop_words]  
    words = [w for w in words if not w in stop_words] 
    return words

In [7]:
#sentence where each word is tokenized
sentences = []
for raw_sentence in raw_sentences:
    if len(raw_sentence) > 0:
        sentences.append(sentence_to_wordlist(raw_sentence))
#print(sentences[1])

In [8]:
token_count = sum([len(sentence) for sentence in sentences])
print("The corpus contains {0:,} tokens".format(token_count))

The corpus contains 81,592,322 tokens


In [9]:
def getWordList(sentences,minCount):
    wordList = []
    count = Counter()
    for sentence in sentences:
        for word in sentence:
            count[word] += 1
    for word,c in count.items():
        if c >= minCount:
            wordList.append(word)
    return wordList

In [10]:
WordList=getWordList(sentences,minCount=5) # for min frequency of a word which should be 5
#print(WordList)

In [11]:
#Vocab Size
len(WordList)

51835

In [12]:
import itertools
ab=list(itertools.chain.from_iterable(sentences))
new_str =' '.join(ab)  
open('wiki2.txt', 'w').write(new_str) # writing clean text to new file

62960183

In [124]:
start_time = time.process_time()
model1 = Word2Vec(sentences,size=300,window=5,min_count=5,seed=72,workers=multiprocessing.cpu_count(),batch_words=1) 
print (time.process_time() - start_time, "seconds")

783.703125 seconds


In [125]:
start_time = time.process_time()
model2 = Word2Vec(sentences,size=300,window=5,min_count=5,seed=82,workers=multiprocessing.cpu_count(),batch_words=1) 
print (time.process_time() - start_time, "seconds")

798.703125 seconds


In [15]:
#word_frequency
w2c = dict()
for item in model1.wv.vocab:
    w2c[item]=model1.wv.vocab[item].count
w2cSorted=dict(sorted(w2c.items(), key=lambda x: x[1],reverse=True))
# reverse output word and their frequency

In [16]:
def Stability(word,similar1,similar2):    
    overlap=0
    for i in range(len(similar1)):
        for j in range(len(similar2)):
            if similar1[i][0] == similar2[j][0]:
                overlap=overlap+1
    return overlap

In [17]:
start_time = time.process_time()
with open('swiki.csv', 'a',errors='replace') as the_file:
    the_file.write('Word,Stability(%),Stability(%) IU,Frequency\n')
    for word in WordList:
        similar1=model1.wv.most_similar(word, topn=10)
        similar2=model2.wv.most_similar(word, topn=10)
        stab=Stability(word,similar1,similar2)
        the_file.write(word+','+str(stab*10)+','+str(stab*100/(20-stab))+','+str(w2cSorted[word])+'\n')
print (time.process_time() - start_time, "seconds")

373.578125 seconds


In [18]:
df=pd.read_csv('swiki.csv')

In [126]:
X = model1[model1.wv.vocab]

<ipython-input-126-5329cf6fff8b>:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  X = model1[model1.wv.vocab]


In [127]:
Y = model2[model2.wv.vocab]

<ipython-input-127-1b77f8722d0e>:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  Y = model2[model2.wv.vocab]


In [128]:
import numpy as np
from sklearn.base import BaseEstimator, ClusterMixin
from sklearn.cluster import DBSCAN
from sklearn.neighbors import kneighbors_graph


def snn(X, neighbor_num, min_shared_neighbor_num):
    """Perform Shared Nearest Neighbor (SNN) clustering algorithm clustering.
    Parameters
    ----------
    X : array or sparse (CSR) matrix of shape (n_samples, n_features), or array of shape (n_samples, n_samples)
    A feature array
    neighbor_num : int
    K number of neighbors to consider for shared nearest neighbor similarity
    min_shared_neighbor_num : int
    Number of nearest neighbors that need to share two data points to be considered part of the same cluster
    """

    # for each data point, find their set of K nearest neighbors
    knn_graph = kneighbors_graph(X, n_neighbors=neighbor_num, include_self=False)
    neighbors = np.array([set(knn_graph[i].nonzero()[1]) for i in range(len(X))])

    # the distance matrix is computed as the complementary of the proportion of shared neighbors between each pair of data points
    snn_distance_matrix = np.asarray([[get_snn_distance(neighbors[i], neighbors[j]) for j in range(len(neighbors))] for i in range(len(neighbors))])

    # perform DBSCAN with the shared-neighbor distance criteria for density estimation
    dbscan = DBSCAN(min_samples=min_shared_neighbor_num, metric="precomputed")
    dbscan = dbscan.fit(snn_distance_matrix)
    return dbscan.core_sample_indices_, dbscan.labels_


def get_snn_similarity(x0, x1):
    """Calculate the shared-neighbor similarity of two sets of nearest neighbors, normalized by the maximum number of shared neighbors"""

    return len(x0.intersection(x1)) / len(x0)


def get_snn_distance(x0, x1):
    """Calculate the shared-neighbor distance of two sets of nearest neighbors, normalized by the maximum number of shared neighbors"""

    return 1 - get_snn_similarity(x0, x1)


class SNN(BaseEstimator, ClusterMixin):
    """Class for performing the Shared Nearest Neighbor (SNN) clustering algorithm.
    Parameters
    ----------
    neighbor_num : int
        K number of neighbors to consider for shared nearest neighbor similarity
    min_shared_neighbor_proportion : float [0, 1]
        Proportion of the K nearest neighbors that need to share two data points to be considered part of the same cluster
    Note: Naming conventions for attributes are based on the analogous ones of DBSCAN
    """

    def __init__(self, neighbor_num, min_shared_neighbor_proportion):

        """Constructor"""

        self.neighbor_num = neighbor_num
        self.min_shared_neighbor_num = round(neighbor_num * min_shared_neighbor_proportion)

    def fit(self, X):

        """Perform SNN clustering from features or distance matrix.
        Parameters
        ----------
        X : array or sparse (CSR) matrix of shape (n_samples, n_features), or array of shape (n_samples, n_samples)
            A feature array
        """

        clusters = snn(X, neighbor_num=self.neighbor_num, min_shared_neighbor_num=self.min_shared_neighbor_num)
        self.core_sample_indices_, self.labels_ = clusters
        if len(self.core_sample_indices_):
            # fix for scipy sparse indexing issue
            self.components_ = X[self.core_sample_indices_].copy()
        else:
            # no core samples
            self.components_ = np.empty((0, X.shape[1]))
        return self

    def fit_predict(self, X, y=None, sample_weight=None):
        """Performs clustering on X and returns cluster labels.
        Parameters
        ----------
        X : array or sparse (CSR) matrix of shape (n_samples, n_features), or \
                array of shape (n_samples, n_samples)
            A feature array, or array of distances between samples if
            ``metric='precomputed'``.
        sample_weight : array, shape (n_samples,), optional
            Weight of each sample, such that a sample with a weight of at least
            ``min_samples`` is by itself a core sample; a sample with negative
            weight may inhibit its eps-neighbor from being core.
            Note that weights are absolute, and default to 1.
        y : Ignored
        Returns
        -------
        y : ndarray, shape (n_samples,)
            cluster labels
        """
        self.fit(X)
        return self.labels_
    

snn1 = SNN(neighbor_num=55, min_shared_neighbor_proportion=0.5)
snn1.fit(X)

snn2 = SNN(neighbor_num=55, min_shared_neighbor_proportion=0.5)
snn2.fit(Y)


C:\Users\anborah\Anaconda3\lib\site-packages\sklearn\base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


SNN(min_shared_neighbor_proportion=None, neighbor_num=55)

In [129]:
core_samples_mask = np.zeros_like(snn1.labels_, dtype=bool)
core_samples_mask[snn1.core_sample_indices_] = True
labelsw1 = snn1.labels_

In [130]:
core_samples_mask2 = np.zeros_like(snn2.labels_, dtype=bool)
core_samples_mask2[snn2.core_sample_indices_] = True
labelsw2 = snn2.labels_

In [ ]:
#adjusted rand score

In [47]:
sklearn.metrics.adjusted_rand_score(labelsw1, labelsw3)

0.8426202511848865